In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Pyspark').getOrCreate()

24/03/28 16:47:28 WARN Utils: Your hostname, Vemulapallis-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.0.107 instead (on interface en1)
24/03/28 16:47:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 16:47:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/28 16:47:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
OrderHeaderDF = spark.read.csv(
        f'AdventureWorks/SalesLT.SalesOrderHeader.csv',
        header=True,
        inferSchema=True
    )

In [6]:
OrderDetailsDF = spark.read.csv(f'AdventureWorks/SalesLT.SalesOrderDetail.csv', header=True, inferSchema=True)

In [7]:
from pyspark.sql.functions import col, isnan, when, count

In [8]:
#Drop columns with all Null Values from OrderHeaderDF

#OrderHeaderDF.select([count(col(c)).alias(c)for c in OrderHeaderDF.columnns].show()
OrderHeaderDF.columns



['SalesOrderID',
 'RevisionNumber',
 'OrderDate',
 'DueDate',
 'ShipDate',
 'Status',
 'OnlineOrderFlag',
 'SalesOrderNumber',
 'PurchaseOrderNumber',
 'AccountNumber',
 'CustomerID',
 'ShipToAddressID',
 'BillToAddressID',
 'ShipMethod',
 'CreditCardApprovalCode',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'Comment',
 'rowguid',
 'ModifiedDate']

In [9]:
OrderHeaderDF.select(count('SalesOrderID')).show()

+-------------------+
|count(SalesOrderID)|
+-------------------+
|                 32|
+-------------------+



In [10]:
OrderHeaderDF.select([count(when (col(c).isNull(),c)).alias(c) for c in OrderHeaderDF.columns]).show()

+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|SalesOrderID|RevisionNumber|OrderDate|DueDate|ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber|AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|ShipMethod|CreditCardApprovalCode|SubTotal|TaxAmt|Freight|TotalDue|Comment|rowguid|ModifiedDate|
+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|           0|             0|        0|      0|       0|     0|              0|               0|                  0|            0|         0|              0|              0|         0|      

In [11]:
OrderHeaderDF.select([count(when (col(c) == 'NULL',c)).alias(c) for c in OrderHeaderDF.columns]).show()

+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|SalesOrderID|RevisionNumber|OrderDate|DueDate|ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber|AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|ShipMethod|CreditCardApprovalCode|SubTotal|TaxAmt|Freight|TotalDue|Comment|rowguid|ModifiedDate|
+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|           0|             0|        0|      0|       0|     0|              0|               0|                  0|            0|         0|              0|              0|         0|      

In [12]:
OrderHeaderDF_NoNulls = OrderHeaderDF.drop("CreditCardApprovalCode", "Comment")

In [13]:
OrderHeaderDF_NoNulls.columns

['SalesOrderID',
 'RevisionNumber',
 'OrderDate',
 'DueDate',
 'ShipDate',
 'Status',
 'OnlineOrderFlag',
 'SalesOrderNumber',
 'PurchaseOrderNumber',
 'AccountNumber',
 'CustomerID',
 'ShipToAddressID',
 'BillToAddressID',
 'ShipMethod',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'rowguid',
 'ModifiedDate']

In [14]:
OrderHeaderDF.select([count(when (col(c) == '00:00.0' ,c)).alias(c) for c in OrderHeaderDF.columns]).show()

+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|SalesOrderID|RevisionNumber|OrderDate|DueDate|ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber|AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|ShipMethod|CreditCardApprovalCode|SubTotal|TaxAmt|Freight|TotalDue|Comment|rowguid|ModifiedDate|
+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|           0|             0|       32|     32|      32|     0|              0|               0|                  0|            0|         0|              0|              0|         0|      

In [15]:
OrderHeaderDF.select([count(when (col(c) == '0' ,c)).alias(c) for c in OrderHeaderDF.columns]).show()

+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|SalesOrderID|RevisionNumber|OrderDate|DueDate|ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber|AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|ShipMethod|CreditCardApprovalCode|SubTotal|TaxAmt|Freight|TotalDue|Comment|rowguid|ModifiedDate|
+------------+--------------+---------+-------+--------+------+---------------+----------------+-------------------+-------------+----------+---------------+---------------+----------+----------------------+--------+------+-------+--------+-------+-------+------------+
|           0|             0|        0|      0|       0|     0|             32|               0|                  0|            0|         0|              0|              0|         0|      

In [16]:
OrderHeaderDF_cleaned = OrderHeaderDF_NoNulls.drop('OrderDate', 'DueDate', 'ShipDate', 'ModifiedDate','OnlineOrderFlag')

In [17]:
OrderHeaderDF_cleaned.show()

+------------+--------------+------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+-----------+---------+---------+----------+--------------------+
|SalesOrderID|RevisionNumber|Status|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|       ShipMethod|   SubTotal|   TaxAmt|  Freight|  TotalDue|             rowguid|
+------------+--------------+------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+-----------+---------+---------+----------+--------------------+
|       71774|             2|     5|         SO71774|        PO348186287|10-4020-000609|     29847|           1092|           1092|CARGO TRANSPORT 5|   880.3484|  70.4279|  22.0087|   972.785|89E42CDC-8506-48A...|
|       71776|             2|     2|         SO71776|      PO19952192051|10-4020-000106|     30072|            640|            640|CARGO TRANSPO

In [18]:
OrderDetailsDF.select(count('SalesOrderID')).show()

+-------------------+
|count(SalesOrderID)|
+-------------------+
|                542|
+-------------------+



In [19]:
OrderDetailsDF.select([count(when (col(c) == 'NULL', c)).alias(c) for c in OrderDetailsDF.columns]).show()

+------------+------------------+--------+---------+---------+-----------------+---------+-------+------------+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal|rowguid|ModifiedDate|
+------------+------------------+--------+---------+---------+-----------------+---------+-------+------------+
|           0|                 0|       0|        0|        0|                0|        0|      0|           0|
+------------+------------------+--------+---------+---------+-----------------+---------+-------+------------+



In [20]:
OrderDetailsDF.select([count(when (col(c) == '00:00.0', c)).alias(c) for c in OrderDetailsDF.columns]).show()

+------------+------------------+--------+---------+---------+-----------------+---------+-------+------------+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal|rowguid|ModifiedDate|
+------------+------------------+--------+---------+---------+-----------------+---------+-------+------------+
|           0|                 0|       0|        0|        0|                0|        0|      0|         542|
+------------+------------------+--------+---------+---------+-----------------+---------+-------+------------+



In [21]:
OrderDetailsDF_cleaned = OrderDetailsDF.drop('ModifiedDate')

In [22]:
#Join OrderDetails and OrderDetailsHeader to get the status of the order.
# Join based on SalesOrderID
SalesOrderStatusDF = OrderHeaderDF_cleaned.join(OrderDetailsDF_cleaned,on='SalesOrderID')

In [23]:
SalesOrderStatusDF.columns


['SalesOrderID',
 'RevisionNumber',
 'Status',
 'SalesOrderNumber',
 'PurchaseOrderNumber',
 'AccountNumber',
 'CustomerID',
 'ShipToAddressID',
 'BillToAddressID',
 'ShipMethod',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'rowguid',
 'SalesOrderDetailID',
 'OrderQty',
 'ProductID',
 'UnitPrice',
 'UnitPriceDiscount',
 'LineTotal',
 'rowguid']

In [24]:
SalesOrderStatusDF = SalesOrderStatusDF.drop('rowguid', 'rowguid')

In [25]:
SalesOrderStatusDF.show()

+------------+--------------+------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+----------+---------+--------+----------+------------------+--------+---------+---------+-----------------+---------+
|SalesOrderID|RevisionNumber|Status|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|       ShipMethod|  SubTotal|   TaxAmt| Freight|  TotalDue|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal|
+------------+--------------+------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+----------+---------+--------+----------+------------------+--------+---------+---------+-----------------+---------+
|       71774|             2|     5|         SO71774|        PO348186287|10-4020-000609|     29847|           1092|           1092|CARGO TRANSPORT 5|  880.3484|  70.4279| 22.0087|   972.785|         

In [26]:
#Check status on current Orders
SalesOrderStatusDF.select(count('SalesOrderID')).show()


+-------------------+
|count(SalesOrderID)|
+-------------------+
|                542|
+-------------------+



In [27]:
#Preparation to add Product details in the SalesOrderStatus

ProductDF = spark.read.csv(f'AdventureWorks/SalesLT.Product.csv', header=True, inferSchema=True)
ProductDF.show()

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------+-----------+----------------+--------------------+----------------------+--------------------+------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|ModifiedDate|
+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------+-----------+----------------+--------------------+----------------------+--------------------+------------+
|      680|HL Road Frame - B...|   FR-R92B-58|Black|     1059.31|   1431.5|  58|1016.04|               18|             6|      00:00.0|       NULL|            NULL|0x474946383961500...|  no_image_availabl...|43DD68D6-14A4-461...|     01:36.8|
|      706|HL Road Frame - R

In [28]:
ProductDF.select(count('ProductID')).show()

+----------------+
|count(ProductID)|
+----------------+
|             295|
+----------------+



In [29]:
ProductDF.select([count(when (col(c) == 'NULL' ,c)).alias(c) for c in ProductDF.columns]).show()

+---------+----+-------------+-----+------------+---------+----+------+-----------------+--------------+-------------+-----------+----------------+--------------+----------------------+-------+------------+
|ProductID|Name|ProductNumber|Color|StandardCost|ListPrice|Size|Weight|ProductCategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|ThumbNailPhoto|ThumbnailPhotoFileName|rowguid|ModifiedDate|
+---------+----+-------------+-----+------------+---------+----+------+-----------------+--------------+-------------+-----------+----------------+--------------+----------------------+-------+------------+
|        0|   0|            0|   50|           0|        0|  84|    97|                0|             0|            0|        197|             295|             0|                     0|      0|           0|
+---------+----+-------------+-----+------------+---------+----+------+-----------------+--------------+-------------+-----------+----------------+--------------+----------

In [30]:
ProductDF.select([count(when (col(c) == '00:00.0' ,c)).alias(c) for c in ProductDF.columns]).show()

+---------+----+-------------+-----+------------+---------+----+------+-----------------+--------------+-------------+-----------+----------------+--------------+----------------------+-------+------------+
|ProductID|Name|ProductNumber|Color|StandardCost|ListPrice|Size|Weight|ProductCategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|ThumbNailPhoto|ThumbnailPhotoFileName|rowguid|ModifiedDate|
+---------+----+-------------+-----+------------+---------+----+------+-----------------+--------------+-------------+-----------+----------------+--------------+----------------------+-------+------------+
|        0|   0|            0|    0|           0|        0|   0|     0|                0|             0|          295|         98|               0|             0|                     0|      0|           0|
+---------+----+-------------+-----+------------+---------+----+------+-----------------+--------------+-------------+-----------+----------------+--------------+----------

In [31]:
ProductDF = ProductDF.drop('DiscontinuedDate', 'SellStartDate', 'SellEndDate')

In [32]:
OrderStatusDF = SalesOrderStatusDF.join(ProductDF, on='ProductID').select(SalesOrderStatusDF['SalesOrderID'], SalesOrderStatusDF['Status'], SalesOrderStatusDF['ProductID'], ProductDF['Name'], ProductDF['Color'])

In [33]:
OrderStatusDF.show()

+------------+------+---------+--------------------+------------+
|SalesOrderID|Status|ProductID|                Name|       Color|
+------------+------+---------+--------------------+------------+
|       71774|     5|      836|ML Road Frame-W -...|      Yellow|
|       71774|     5|      822|ML Road Frame-W -...|      Yellow|
|       71776|     2|      907|         Rear Brakes|      Silver|
|       71780|     1|      905|ML Mountain Frame...|      Silver|
|       71780|     1|      983|Mountain-400-W Si...|      Silver|
|       71780|     1|      988|Mountain-500 Silv...|      Silver|
|       71780|     1|      748|HL Mountain Frame...|      Silver|
|       71780|     1|      990|Mountain-500 Blac...|       Black|
|       71780|     1|      926|LL Mountain Frame...|       Black|
|       71780|     1|      743|HL Mountain Frame...|       Black|
|       71780|     1|      782|Mountain-200 Blac...|       Black|
|       71780|     1|      918|LL Mountain Frame...|      Silver|
|       71

In [35]:
#checking the status of a salesorder with salesOrderID = 71774
OrderStatusDF.filter(OrderStatusDF.SalesOrderID == 71774).show()

+------------+------+---------+--------------------+------+
|SalesOrderID|Status|ProductID|                Name| Color|
+------------+------+---------+--------------------+------+
|       71774|     5|      836|ML Road Frame-W -...|Yellow|
|       71774|     5|      822|ML Road Frame-W -...|Yellow|
+------------+------+---------+--------------------+------+

